In [4]:
import os
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
import glob
import copy
import re
import string

from nltk.corpus import stopwords
stoplist = set(stopwords.words("english"))

from sklearn.model_selection import train_test_split
import tensorflow as tf

# Import text file

In [2]:
# absolute path to search all text files inside the folder
path = r'D:\Coding\Twitter_Sentimental_Analysis\Training_Data/*.txt'
files = glob.glob(path)
print(files)

['D:\\Coding\\Twitter_Sentimental_Analysis\\Training_Data\\SemEval2017-task4-dev.subtask-A.english.INPUT.txt', 'D:\\Coding\\Twitter_Sentimental_Analysis\\Training_Data\\SemEval2017-task4-dev.subtask-BD.english.INPUT.txt', 'D:\\Coding\\Twitter_Sentimental_Analysis\\Training_Data\\SemEval2017-task4-dev.subtask-CE.english.INPUT.txt', 'D:\\Coding\\Twitter_Sentimental_Analysis\\Training_Data\\twitter-2013dev-A.txt', 'D:\\Coding\\Twitter_Sentimental_Analysis\\Training_Data\\twitter-2013test-A.txt', 'D:\\Coding\\Twitter_Sentimental_Analysis\\Training_Data\\twitter-2013train-A.txt', 'D:\\Coding\\Twitter_Sentimental_Analysis\\Training_Data\\twitter-2014sarcasm-A.txt', 'D:\\Coding\\Twitter_Sentimental_Analysis\\Training_Data\\twitter-2014test-A.txt', 'D:\\Coding\\Twitter_Sentimental_Analysis\\Training_Data\\twitter-2015test-A.txt', 'D:\\Coding\\Twitter_Sentimental_Analysis\\Training_Data\\twitter-2015testBD.txt', 'D:\\Coding\\Twitter_Sentimental_Analysis\\Training_Data\\twitter-2015train-A.txt',

In [3]:
# Create a new dictionary and loop all text files as dataframes into it
dataframes = {}

for i, file in enumerate(files):
    df = pd.read_csv(files[i], delimiter = "\t", header=None)
    dataframes[i] = df

word_list = [
    "neutral",
    "negative",
    "-1",
    "positive",
    "0",
    "1"]

In [4]:
# Create an empty DataFrame
train_df = pd.DataFrame(columns=['Sentiment', 'Text'])

# Loop through all dataframes, and only take the column with sentiment and text!!
for i in range(len(dataframes)):
    for index, value in enumerate(dataframes[i].iloc[0,:]):
        if str(value) in word_list:
            
            # Create a temporary DataFrame with the two Series
            temp_df = pd.DataFrame({'Sentiment': dataframes[i].iloc[:,index], 'Text': dataframes[i].iloc[:,index+1]})

            # Append the temporary DataFrame to the original DataFrame
            train_df = train_df.append(temp_df, ignore_index=True)


C:\Users\amito\AppData\Local\Temp\ipykernel_16936\2705102758.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(temp_df, ignore_index=True)


In [5]:
# Save the DataFrame as a CSV file for viewing
train_df.to_csv('train_df.csv', sep=' ', index=False)

In [ ]:
for index, row_value in enumerate(train_df['Sentiment']):
    # print(index)
    try:
        # Check the value and perform operations accordingly
        if row_value == 'positive' or str(row_value) == str(2) or row_value == 2:
            train_df.at[index, 'Sentiment'] = 1
        elif row_value == 'negative' or str(row_value) == str(-2) or row_value == -2:
            train_df.at[index, 'Sentiment'] = -1
        elif row_value == 'neutral' or row_value == 0 or str(row_value) == str(0) or row_value == 'off topic':
            train_df = train_df.drop(index)
    except KeyError:
        print("INDEX: ", str(index))


In [7]:
# Save the DataFrame as a CSV file for viewing
train_df.to_csv('train_df_sentiment_all_numbers.csv', sep=' ', index=False)

In [5]:
# Load saved csv to df
train_df = pd.read_csv('train_df_sentiment_all_numbers.csv', sep=' ')

# Pre-processing text

In [6]:
train_df = train_df.sort_values(by=['Sentiment'])
train_df = train_df.reset_index()
train_df = train_df.drop(columns = ['index'])
train_df.head()

,Sentiment,Text
0,-1,If these runway renovations at the airport pre...
1,-1,@twats0n what are you some stupid Twilight vam...
2,-1,"@maricruzannette first time I heard of ""twilig..."
3,-1,@TerryGilliam Sat my daughter down and we watc...
4,-1,@FoxNews This administratio is living in the t...


In [7]:
train_df['Sentiment'].value_counts()

 1    71986
-1    23420
Name: Sentiment, dtype: int64

In [8]:
balanced_train_df = copy.deepcopy(train_df[0:23420*2])

In [9]:
balanced_train_df['Sentiment'].value_counts()

-1    23420
 1    23420
Name: Sentiment, dtype: int64

### Processing text

In [10]:
def preprocess_text(text):
    # Remove http / https links
    text = re.sub(r'http\S+|https\S+', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove stopwords
    text = ' '.join(word for word in text.split() if word not in stoplist)
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [11]:
def preprocess_sent(text):
    if text == -1:
        text = 0
    return text

In [12]:
balanced_train_df['Text'] = balanced_train_df['Text'].apply(preprocess_text)
balanced_train_df['Sentiment'] = balanced_train_df['Sentiment'].apply(preprocess_sent)

In [13]:
balanced_train_df

,Sentiment,Text
0,0,runway renovations airport prevent seeing tayl...
1,0,twatsn stupid twilight vampire real ones dont ...
2,0,maricruzannette first time heard twilight assu...
3,0,terrygilliam sat daughter watched monkeys chan...
4,0,foxnews administratio living twilight zone may...
...,...,...
46835,1,gonna record ciarans mash fleetwood mac songs ...
46836,1,pearl may bug shit right fleetwood mac vinyl
46837,1,friday night got new red lipstick put nowhere ...
46838,1,friday nite lineup recommences tonight p pstfl...


In [14]:
# Save the DataFrame as a CSV file for viewing
balanced_train_df.to_csv('balanced_train_df.csv', sep=' ', index=False)

In [14]:
# Load saved csv to df
balanced_train_df = pd.read_csv('balanced_train_df.csv', sep=' ')

In [18]:
print(balanced_train_df.Text.str.len().max())

3873
